## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import ipywidgets as widgets
from config import gkey
gmaps.configure(api_key = gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

In [4]:
while True: # While loop to validate for temperature inputs
    inp = input("What is the coldest temperature you would like to have on your trip? ")
    try:
        float(inp)
        break
    except ValueError:
        print("This is not a valid input.")
        continue

temp_min = float(inp) # Testing with 35

In [5]:
while True: # While loop to validate for temperature inputs
    inp = input("What is the hottest temperature you would like to have on your trip? ")
    try:
        float(inp)
        break
    except ValueError:
        print("This is not a valid input.")
        continue

temp_max = float(inp) # Testing with 87

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] >= temp_min) & (city_data_df["Max Temp"] <= temp_max)]

In [7]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Augusta,AU,-32.5000,137.7667,65.71,72,47,18.61,scattered clouds
1,1,Karratha,AU,-20.7377,116.8463,82.47,74,13,11.12,few clouds
2,2,Faya,SA,18.3851,42.4509,61.56,62,73,4.12,broken clouds
3,3,Kahului,US,20.8947,-156.4700,74.23,62,13,5.19,few clouds
4,4,Bengkulu,ID,-3.8004,102.2655,75.70,89,99,3.67,moderate rain


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Augusta,AU,65.71,scattered clouds,-32.5000,137.7667,
1,Karratha,AU,82.47,few clouds,-20.7377,116.8463,
2,Faya,SA,61.56,broken clouds,18.3851,42.4509,
3,Kahului,US,74.23,few clouds,20.8947,-156.4700,
4,Bengkulu,ID,75.70,moderate rain,-3.8004,102.2655,
5,Hamilton,US,76.75,clear sky,39.1834,-84.5333,
6,Jamestown,US,57.85,overcast clouds,42.0970,-79.2353,
8,Rikitea,PF,79.86,few clouds,-23.1203,-134.9692,
9,Bredasdorp,ZA,71.28,clear sky,-34.5322,20.0403,
10,Kapaa,US,77.11,clear sky,22.0752,-159.3190,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Storing new lat and lng of the hotel.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

        # Added these replaces to ensure that lat/lng coords would be the location of the hotels
        # Original lat/lng coords were sometimes in the ocean, making Devlierable 3 API result in AssertionError
        hotel_df.loc[index, "Lat"] = hotels["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = hotels["results"][0]["geometry"]["location"]["lng"]
    except IndexError:
        hotel_df["Hotel Name"].replace("", np.nan, inplace = True)    

In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))